<a href="https://colab.research.google.com/github/laumojo10/Aplicativo-Web-Doctores-y-Pacientes/blob/main/Proyectos1Organizaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Proyecto Creación de Bot texto en negrita**

In [81]:


!pip install ccxt requests yfinance ta pandas numpy scipy torch matplotlib seaborn plotly python-binance

In [82]:
import ccxt
import requests
import time
#import yfinance  as yf
import ta
import pandas as pd
import numpy  as np
from scipy.optimize import minimize
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
#from binance.client import Client

In [196]:
from binance.client import Client

In [197]:
# Configure your API keys (obténlas desde Binance)
API_KEY = "A0qBhK6SlZkAEty05D4Oj16L4PAdCsCiTpUqkb4DbeT6leDv3ofIZOxpSU20Ps6z"
API_SECRET = "R04BSkajiI4LrwbBy3nVKkXhatOvygyxIh8DyQ01IAto1jUPEeiIomk7nWfVfy59"

In [198]:
# Inicializa el cliente de Binance
client = Client(API_KEY, API_SECRET,tld="us")

In [199]:
# Conect with Binance- Library CCXT (python)
exchange = ccxt.binance()


In [203]:
#Historical data in the interval  1 day , with the limit 200
def get_historical_data(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_1DAY, limit=200):
    try:
        klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    except Exception as e:
        print(f"Error al obtener datos de Binance: {e}")
        return None  # Retorna None en caso de error

    # Convertir a DataFrame
    df = pd.DataFrame(klines, columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume',
                                       'CloseTime', 'QuoteVolume', 'Trades', 'TakerBaseVol', 'TakerQuoteVol', 'Ignore'])

    # Convertir el precio de cierre a float y la marca de tiempo a formato legible
    df['Close_day'] = df['Close'].astype(float)
    df['Time'] = pd.to_datetime(df['Time'], unit='ms')

    return df[['Time', 'Close_day']]


In [204]:
df = get_historical_data()  # Call the function to populate df

# Now you can access 'Close_day'
print(df['Close_day'])

0      65392.96
1      61536.09
2      60690.47
3      58156.48
4      53944.07
         ...   
195    97712.99
196    96550.39
197    97477.75
198    97571.43
199    97655.83
Name: Close_day, Length: 200, dtype: float64


In [205]:
# Fetch BTC/USDT historical klines with a 4-hour interval
def get_historical_data(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_4HOUR, limit=200):
    try:
        klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    except Exception as e:
        print(f"Error al obtener datos de Binance: {e}")
        return None  # Retorna None en caso de error

    # Convertir a DataFrame
    df = pd.DataFrame(klines, columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume',
                                       'CloseTime', 'QuoteVolume', 'Trades', 'TakerBaseVol', 'TakerQuoteVol', 'Ignore'])

    # Convertir la columna 'Close' a flotante
    df['Close_hour'] = df['Close'].astype(float)

    # Convertir la columna 'Time' a formato datetime
    df['Time'] = pd.to_datetime(df['Time'], unit='ms')

    return df[['Time', 'Close_hour']]


In [206]:
df = get_historical_data()
print(df.head())

                 Time  Close_hour
0 2025-01-13 20:00:00    94443.68
1 2025-01-14 00:00:00    94962.91
2 2025-01-14 04:00:00    94813.42
3 2025-01-14 08:00:00    96447.02
4 2025-01-14 12:00:00    96681.52


In [207]:
# Fetch BTC/USDT historical klines with a 1 week interval
def get_historical_data(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_1WEEK, limit=200):
    try:
        klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    except Exception as e:
        print(f"Error al obtener datos de Binance: {e}")
        return None  # Retorna None en caso de error

    # Convertir a DataFrame
    df = pd.DataFrame(klines, columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume',
                                       'CloseTime', 'QuoteVolume', 'Trades', 'TakerBaseVol', 'TakerQuoteVol', 'Ignore'])

    # Convertir la columna 'Close' a flotante
    df['Close_1WEEK'] = df['Close'].astype(float)

    # Convertir la columna 'Time' a formato datetime
    df['Time'] = pd.to_datetime(df['Time'], unit='ms')

    print(df[['Time', 'Close_1WEEK']])  # Muestra los datos en consola

    return df[['Time', 'Close_1WEEK']]


In [208]:
df = get_historical_data()
print(df.head())

          Time  Close_1WEEK
0   2021-04-19     49063.82
1   2021-04-26     56577.85
2   2021-05-03     58235.83
3   2021-05-10     46431.10
4   2021-05-17     34646.69
..         ...          ...
195 2025-01-13    101032.19
196 2025-01-20    102618.04
197 2025-01-27     97692.60
198 2025-02-03     96426.86
199 2025-02-10     97655.83

[200 rows x 2 columns]
        Time  Close_1WEEK
0 2021-04-19     49063.82
1 2021-04-26     56577.85
2 2021-05-03     58235.83
3 2021-05-10     46431.10
4 2021-05-17     34646.69


In [209]:
def calculate_sma(df, period=200):
    df[f'SMA{period}'] = df['Close'].rolling(window=period).mean()
    return df

In [210]:

def get_last_SMA200(df):

    if len(df) < 2:
        print("Error: No hay suficientes datos para calcular SMA200.")
        return None  # Retorna None si no hay suficientes datos

    # Extraer los valores de cierre de las últimas dos semanas
    last_SMA200 = df['Close'].iloc[-2]  # Penúltima semana
    for_last_SMA200 = df['Close'].iloc[-1]  # Última semana

    return pd.DataFrame({'Semana': ['Penúltima', 'Última'], 'Cierre': [last_SMA200, for_last_SMA200]})

In [216]:
def obtener_semanas(df):
    last_SMA200 = df['Close_1WEEK'].iloc[-2]
    for_last_SMA200 = df['Close_1WEEK'].iloc[-1]


In [217]:
df = obtener_semanas()
print(df.head())

TypeError: obtener_semanas() missing 1 required positional argument: 'df'

In [154]:
for_last_SMA200=df['Close_1WEEK'].iloc[-1]
print(for_last_SMA200)

KeyError: 'Close_1WEEK'

In [142]:
last_SMA200=df['Close_1WEEK'].iloc[-2]
print(inicio)

KeyError: 'Close_1WEEK'

In [155]:
# Sample data: Assume you have a DataFrame with a 'Close' column
bull_market_criteria = (last_SMA200 - for_last_SMA200)/for_last_SMA200
print(bull_market_criteria)

alpha_bull=0.01
if bull_market_criteria > alpha_bull:
    print("bull market")
else:
    print("bear market")

NameError: name 'last_SMA200' is not defined

In [ ]:
ff =df['Close_1WEEK'].iloc[-1]
f2 = df['200_MA'].iloc[-1]
print(ff)
print(f2)

97429.06
44182.25119999999


In [ ]:
criteria_2 = (ff -f2)/f2
alpha_bull_2= 0.01
if criteria_2 >alpha_bull_2:
    print("bull market")
else:
    print("bear market")

bull market


DCA

In [ ]:
#DCA: dollar cost average
#constant investing quantities in regular periods of time


In [ ]:

    # Obtener el precio actual de Bitcoin (BTC/USDT)
    price = client.get_symbol_ticker(symbol="BTCUSDT")["price"]
    price = float(price)
    print(f"Precio actual de BTC/USDT: {price}")
    # Esperar 1 minuto (60 segundos)


Precio actual de BTC/USDT: 97429.06


In [ ]:
df_precio=pd.DataFrame([[price]],columns=['price_Actual'])

In [ ]:
precio_Actual=df_precio['price_Actual'].iloc[0]
print(precio_Actual)

97429.06


In [ ]:
print(precio_Actual)
print(price_lastday_close)


97429.06
97363.62


In [ ]:
x = (precio_Actual - price_lastday_close)/price_lastday_close
alpha_Day = -0.02
if x < alpha_Day:
  print("investment ok")
else:
  print("investment not ok")

investment not ok
